In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.14.0 pyspark-shell'
import re
import sys
from operator import add
from typing import Iterable,Tuple
from pyspark.resultiterable import ResultIterable
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType,FloatType
from pyspark.streaming import StreamingContext

In [9]:
## Emitter Part(start pagerank algorithm right before start this chunk using p1t3q8.ipynb)

spark = SparkSession.builder.getOrCreate()

scc = StreamingContext(spark.sparkContext,1)
emitter_stream = scc.textFileStream("gs://csee4121homework/outputs/p2t2_whole.csv/")

def function(emitter_stream):
    df = emitter_stream.map(lambda content: content.split('\t'))\
                    .toDF(['_c0', '_c1'])\
                    .coalesce(1).write.option("delimiter", "\t")\
                    .mode("append").csv("gs://csee4121homework/outputs/p2t2_emitter/")
emitter_stream.foreachRDD(function)
scc.start()

22/05/02 18:05:45 WARN org.apache.spark.streaming.StreamingContext: Dynamic Allocation is enabled for this application. Enabling Dynamic allocation for Spark Streaming applications can cause data loss if Write Ahead Log is not enabled for non-replayable sources. See the programming guide for details on how to enable the Write Ahead Log.
22/05/02 18:05:46 ERROR org.apache.spark.streaming.scheduler.JobScheduler: Error running job streaming job 1651514746000 ms.0
org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/lib/spark/python/pyspark/streaming/util.py", line 68, in call
    r = self.func(t, *rdds)
  File "/usr/lib/spark/python/pyspark/streaming/dstream.py", line 155, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "/tmp/ipykernel_723/2416125911.py", line 9, in function
    df = emitter_stream.map(lambda content: content.split('\t'))\
  File "/usr/lib/spark/python/pyspark/sql/session.py", line 66, in toDF
    r

In [10]:
## Receiver Part
spark = SparkSession.builder.getOrCreate()
userSchema =  StructType([ StructField("_c0", StringType(), True),\
                          StructField("_c1", FloatType(), True)])

stream = spark.readStream \
    .option("delimiter", "\t") \
    .schema(userSchema) \
    .csv("gs://csee4121homework/outputs/p2t2_emitter/") 

receiver_path_checkpoint = 'gs://csee4121homework/outputs/whole_checkpoint/'

path2 = 'gs://csee4121homework/outputs/whole_receiver_from_emitter/'

receiver = stream\
    .select("_c0", "_c1")\
    .where("_c1 > 0.5")\
    .coalesce(1)\
    .writeStream \
    .format("csv") \
    .option("mode","overwrite")\
    .option("delimiter", "\t") \
    .option("checkpointLocation", receiver_path_checkpoint)\
    .option("path", path2) \
    
receiver.start()

22/05/02 18:06:16 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


22/05/02 18:06:32 ERROR org.apache.spark.streaming.scheduler.JobScheduler: Error generating jobs for time 1651514792000 ms
org.apache.hadoop.mapreduce.lib.input.InvalidInputException: Input path does not exist: gs://csee4121homework/outputs/p2t2_whole.csv/_SUCCESS
	at org.apache.hadoop.mapred.LocatedFileStatusFetcher.getFileStatuses(LocatedFileStatusFetcher.java:153)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.listStatus(FileInputFormat.java:280)
	at org.apache.hadoop.mapreduce.lib.input.FileInputFormat.getSplits(FileInputFormat.java:396)
	at org.apache.spark.rdd.NewHadoopRDD.getPartitions(NewHadoopRDD.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:300)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:296)
	at org.apache.spark.streaming.dstream.FileInputDStream.$anonfun$filesToRDD$1(FileInputDStream.scala:289)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.coll

In [12]:
scc.stop()

22/05/02 18:10:47 WARN org.apache.spark.streaming.StreamingContext: StreamingContext has already been stopped


In [14]:
receiver.stop()

In [20]:
spark = SparkSession.builder.getOrCreate()
print("Number of articles in the database has a rank greater than 0.5 for part 2 task 1:",\
      spark.read.csv("gs://csee4121homework/outputs/whole_receiver",sep = "\t").count())
print("Number of articles in the database has a rank greater than 0.5 for part 2 task 2:",\
      spark.read.csv("gs://csee4121homework/outputs/whole_receiver_from_emitter",sep = "\t").repartition(1).count())

Number of articles in the database has a rank greater than 0.5 for part 2 task 1: 1762190
Number of articles in the database has a rank greater than 0.5 for part 2 task 2: 1762190
